# Ingesting data into Filess Cloud based Postgres Database

## Testing the connection

In [ ]:
import psycopg2
import pandas as pd 

hostname = "bpd9p.h.filess.io"
database = "database_wheneverof"
port = "5432"
username = "database_wheneverof"
password = "b2bf3de4d74d8d014fc7370f7b911feeba371894"

# Establishing the connection
conn = psycopg2.connect(
   database=database, user=username, password=password, host=hostname, port=port
)
# Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ", data)

# Closing the connection
conn.close()


Connection established to:  ('PostgreSQL 14.4 on aarch64-unknown-linux-musl, compiled by gcc (Alpine 11.2.1_git20220219) 11.2.1 20220219, 64-bit',)


## Uploading the data into postgress

In [ ]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# PostgreSQL connection details
hostname = "bpd9p.h.filess.io"
database = "database_wheneverof"
port = 5432
username = "database_wheneverof"
password = "b2bf3de4d74d8d014fc7370f7b911feeba371894"

# CSV file path
csv_file_path = "data/data.csv"  # <- Replace with your actual file path
table_name = "machine_data"
schema_name = "myschema"  # <- You can rename this schema if you want

# Load the CSV into pandas
df = pd.read_csv(csv_file_path)

# Clean column names to be SQL-safe
df.columns = [col.strip().lower().replace(" ", "_").replace("[", "").replace("]", "") for col in df.columns]

# Build CREATE TABLE statement based on DataFrame structure
def generate_create_table(df, full_table_name):
    sql_fields = []
    for col in df.columns:
        if df[col].dtype == "int64":
            col_type = "BIGINT"
        elif df[col].dtype == "float64":
            col_type = "FLOAT"
        else:
            col_type = "TEXT"
        sql_fields.append(f"{col} {col_type}")
    return f"CREATE TABLE {full_table_name} ({', '.join(sql_fields)});"

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port
    )
    cursor = connection.cursor()
    print("✅ Connected to PostgreSQL!")

    # Step 1: Create your schema if it doesn't exist
    cursor.execute(sql.SQL("CREATE SCHEMA IF NOT EXISTS {} AUTHORIZATION {}").format(
        sql.Identifier(schema_name),
        sql.Identifier(username)
    ))
    print(f"🏗️ Created/confirmed schema `{schema_name}`.")

    # Step 2: Drop table if exists
    cursor.execute(sql.SQL("DROP TABLE IF EXISTS {}.{}").format(
        sql.Identifier(schema_name),
        sql.Identifier(table_name)
    ))
    print(f"🧹 Dropped table `{schema_name}.{table_name}` if it existed.")

    # Step 3: Create table
    full_table_name = f"{schema_name}.{table_name}"
    create_sql = generate_create_table(df, full_table_name)
    cursor.execute(create_sql)
    print(f"📦 Created table `{full_table_name}`.")

    # Step 4: Insert data in batches
    batch_size = 500
    total_records = len(df)
    print(f"⬆️ Inserting {total_records} records...")

    for start in range(0, total_records, batch_size):
        end = min(start + batch_size, total_records)
        batch = df.iloc[start:end]

        columns = ', '.join(batch.columns)
        values = ', '.join(['%s'] * len(batch.columns))
        insert_sql = f"INSERT INTO {full_table_name} ({columns}) VALUES ({values})"

        records = [tuple(x) for x in batch.to_numpy()]
        cursor.executemany(insert_sql, records)
        connection.commit()
        print(f"✅ Inserted rows {start + 1} to {end}")

    print("🎉 All data inserted successfully into `{full_table_name}`!")

except Exception as e:
    print("❌ Error:", e)

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
        print("🔒 PostgreSQL connection closed.")


✅ Connected to PostgreSQL!
🏗️ Created/confirmed schema `myschema`.
🧹 Dropped table `myschema.machine_data` if it existed.
📦 Created table `myschema.machine_data`.
⬆️ Inserting 10000 records...
✅ Inserted rows 1 to 500
✅ Inserted rows 501 to 1000
✅ Inserted rows 1001 to 1500
✅ Inserted rows 1501 to 2000
✅ Inserted rows 2001 to 2500
✅ Inserted rows 2501 to 3000
✅ Inserted rows 3001 to 3500
✅ Inserted rows 3501 to 4000
✅ Inserted rows 4001 to 4500
✅ Inserted rows 4501 to 5000
✅ Inserted rows 5001 to 5500
✅ Inserted rows 5501 to 6000
✅ Inserted rows 6001 to 6500
✅ Inserted rows 6501 to 7000
✅ Inserted rows 7001 to 7500
✅ Inserted rows 7501 to 8000
✅ Inserted rows 8001 to 8500
✅ Inserted rows 8501 to 9000
✅ Inserted rows 9001 to 9500
✅ Inserted rows 9501 to 10000
🎉 All data inserted successfully into `{full_table_name}`!
🔒 PostgreSQL connection closed.
